<a href="https://colab.research.google.com/github/Jung-Yong-Hun/Stock_Forecast/blob/main/1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import requests
headers = {"User-Agent": "Mozilla/5.0"}

# WSJ 헤드라인 섹션 가져옴
webpage = requests.get("https://www.wsj.com/news/economy?mod=nav_top_section", headers=headers)

# 나스닥 종합주가지수
webpage_nasdaq = requests.get("https://kr.investing.com/indices/nasdaq-composite", headers=headers)

In [42]:
from bs4 import BeautifulSoup

In [43]:
# WSJ 헤드라인
headline_str = ''
bs = BeautifulSoup(webpage.text, 'html.parser')

In [44]:
import re

# 나스닥 지수
nasdaq_indices = ''
bs_nasdaq = BeautifulSoup(webpage_nasdaq.text, 'html.parser')
nasdaq_indices = bs_nasdaq.find("span", {"class" : "instrument-price_change-percent__19cas"})
#print(nasdaq_indices)
#print(nasdaq_indices.text)

#불필요문자 제거
table = str.maketrans("()%", "   ")

indices = float(nasdaq_indices.text.translate(table))
indices

1.67

In [45]:
#헤드라인 텍스트 가져오기
headline_list = bs.select('img')
headline_list

headline_str = ''

for i in range(0, len(headline_list)):
    if(headline_list[i].has_attr('title')):
        print('[ '+str(i)+ ' ] 번째 헤드라인 '+headline_list[i]['title'])
        headline_str = headline_str + str(i+1)+') ' + headline_list[i]['title'] + ' '

print(headline_str)

[ 0 ] 번째 헤드라인 The Labor Shortage Is Here to Stay. Businesses Are Adjusting.
[ 1 ] 번째 헤드라인 Tight Labor Market, Supply Constraints Point to Persistent Inflation
[ 2 ] 번째 헤드라인 Mass Evictions Didn’t Result After U.S. Ban Ended, Despite Fears
[ 3 ] 번째 헤드라인 Accelerating Inflation Spreads Through the Economy
[ 4 ] 번째 헤드라인 China’s Factory-Gate Prices Rise at Record Pace
[ 5 ] 번째 헤드라인 Fed Worried About Inflation Risk as It Firmed Up Tapering Plan
[ 6 ] 번째 헤드라인 How Retail Cheese Prices Weather Market Volatility
[ 7 ] 번째 헤드라인 How the Pandemic Is Accelerating Changes in Your Packaging 
[ 8 ] 번째 헤드라인 ‘Dark Stores’: How Groceries Are Being Delivered in Minutes
1) The Labor Shortage Is Here to Stay. Businesses Are Adjusting. 2) Tight Labor Market, Supply Constraints Point to Persistent Inflation 3) Mass Evictions Didn’t Result After U.S. Ban Ended, Despite Fears 4) Accelerating Inflation Spreads Through the Economy 5) China’s Factory-Gate Prices Rise at Record Pace 6) Fed Worried About Inflation Risk

In [46]:
# client ID : eqwc_HUTOA7373MjFNQ6
# secret key : BEyf9Pzttu

import os
import sys
import urllib.request

client_id = "eqwc_HUTOA7373MjFNQ6"
client_secret = "BEyf9Pzttu"
encText = urllib.parse.quote(headline_str)
data = "source=en&target=ko&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-id", client_id)
request.add_header("X-naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()

if(rescode==200):
    response_body =response.read()
    print(response_body.decode("utf-8"))
else:
    print('Error Code:' + rescode)



{"message":{"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0","result":{"srcLangType":"en","tarLangType":"ko","translatedText":"1) 노동력 부족은 여전하다. 기업 조정 중 2) 노동시장 긴축, 지속적인 인플레이션 공급 제약 3) 미국의 금지가 끝난 후에도 대량 퇴거가 발생하지 않음4) 경제를 통한 인플레이션 확산 가속화5) 중국 공장-게이트 가격 상승 우려 6)l 치즈 가격 날씨 시장 변동성8) 포장의 변화를 가속화하는 방법 9) '암매장': 식료품이 몇 분 만에 배달되는 방법","engineType":"UNDEF_MULTI_SENTENCE","pivot":null}}}


In [47]:
text = response_body.decode('utf-8')

import json
parser = json.loads(text)
text = parser['message']['result']['translatedText']
print(text)

1) 노동력 부족은 여전하다. 기업 조정 중 2) 노동시장 긴축, 지속적인 인플레이션 공급 제약 3) 미국의 금지가 끝난 후에도 대량 퇴거가 발생하지 않음4) 경제를 통한 인플레이션 확산 가속화5) 중국 공장-게이트 가격 상승 우려 6)l 치즈 가격 날씨 시장 변동성8) 포장의 변화를 가속화하는 방법 9) '암매장': 식료품이 몇 분 만에 배달되는 방법


In [48]:
print(re.compile('[가-힣]+').findall(text))

['노동력', '부족은', '여전하다', '기업', '조정', '중', '노동시장', '긴축', '지속적인', '인플레이션', '공급', '제약', '미국의', '금지가', '끝난', '후에도', '대량', '퇴거가', '발생하지', '않음', '경제를', '통한', '인플레이션', '확산', '가속화', '중국', '공장', '게이트', '가격', '상승', '우려', '치즈', '가격', '날씨', '시장', '변동성', '포장의', '변화를', '가속화하는', '방법', '암매장', '식료품이', '몇', '분', '만에', '배달되는', '방법']


In [49]:
good_words = ['최고','혜택','증가','상승','원조','수익','기대감','높은','회복세','회복']
good_words_count = 0

bad_words = ['최악','침체','부채','코로나','감소','사망','사고','트럼프','위험','인플레이션','엘살바도르','경고','Covid-19','긴축']
bad_words_count =0

In [50]:
def get_translate(text, lan):
    data = {'text' : text,
            'source' : 'en',
            'target' : lan
            }
    url = "https://openapi.naver.com/v1/papago/n2mt"
    header = {"X-Naver-Client-id" : client_id,
          "X-naver-Client-Secret" : client_secret}

    response = requests.post(url, headers=header, data=data)
        # response code : 200 정상 , 나머지는 오류
    rescode = response.status_code

    if(rescode == 200):
        text_data = response.json()
        print(text_data['message']['result']['translatedText'])
    else:
        print('Error code: ', rescode)
    return text_data['message']['result']['translatedText']

In [51]:
translate_list = []

for i in range(len(headline_list)):

    # 번역된 텍스트
    kor_text = get_translate(headline_list[i]['title'], 'ko')
    translate_list.append(kor_text)

    # good words 존재 여부
    for j in range(0, len(good_words)):
        if(good_words[j] in kor_text):
            print('-------------')
            print("good words : ", good_words[j])
            print("-------------")
            good_words_count = good_words_count + 1
    # bad words 존재 여부
    for k in range(0, len(bad_words)):
        if(bad_words[k] in kor_text):
            print('-------------')
            print("bad words : ", bad_words[k])
            print("-------------")
            bad_words_count = bad_words_count + 1

노동력 부족은 지속될 것이다. 기업이 적응하고 있다.
노동시장 긴축, 공급 제약, 지속적인 인플레이션 요인
-------------
bad words :  인플레이션
-------------
-------------
bad words :  긴축
-------------
미국의 반이 끝난 후에도 두려움에도 불구하고 대량 퇴거가 일어나지 않았다.
경제 전반에 걸쳐 인플레이션 확산
-------------
bad words :  인플레이션
-------------
기록적인 속도로 상승하는 중국 공장 게이트 가격
-------------
good words :  상승
-------------
연방준비제도이사회(FRB)가 테이퍼링 계획을 추진하면서 인플레이션 위험 우려
-------------
bad words :  위험
-------------
-------------
bad words :  인플레이션
-------------
소매치즈 가격 변동성 예측 방법
전염병이 포장의 변화를 가속화하는 방법
'어두운 상점': 식료품이 몇 분 만에 배달되는 방법


In [52]:
print(good_words_count)
print(bad_words_count)
print(indices)

1
5
1.67


In [53]:
import pandas as pd
from datetime import datetime
print(datetime.today().strftime("%Y-%m-%d"))


2021-10-14


In [54]:
columns = ['date', 'good_words_count', 'bad_words_count', 'indices']
df_total = pd.DataFrame(columns=columns)
df_total.loc[1] = [datetime.today().strftime("%Y-%m-%d"), good_words_count, bad_words_count, indices]
df_total

,date,good_words_count,bad_words_count,indices
1,2021-10-14,1,5,1.67


In [55]:
from google.colab import drive
drive.mount('/content.drive')

# /content.drive/MyDrive/주가지수_1014

df_total.to_csv('/content.drive/MyDrive/주가지수_1014/'+datetime.today().strftime("%Y-%m-%d")+'_indice.csv', encoding='utf-8')

Drive already mounted at /content.drive; to attempt to forcibly remount, call drive.mount("/content.drive", force_remount=True).


In [56]:
columns = ['eng_text', 'kor_text']
df = pd.DataFrame(columns=columns)

for i in range(0, len(translate_list)):
    kor_text = translate_list[i]
    eng_text = headline_list[i]['title']
    df.loc[i+1] = [eng_text,kor_text]

In [57]:
df

,eng_text,kor_text
1,The Labor Shortage Is Here to Stay. Businesses...,노동력 부족은 지속될 것이다. 기업이 적응하고 있다.
2,"Tight Labor Market, Supply Constraints Point t...","노동시장 긴축, 공급 제약, 지속적인 인플레이션 요인"
3,Mass Evictions Didn’t Result After U.S. Ban En...,미국의 반이 끝난 후에도 두려움에도 불구하고 대량 퇴거가 일어나지 않았다.
4,Accelerating Inflation Spreads Through the Eco...,경제 전반에 걸쳐 인플레이션 확산
5,China’s Factory-Gate Prices Rise at Record Pace,기록적인 속도로 상승하는 중국 공장 게이트 가격
6,Fed Worried About Inflation Risk as It Firmed ...,연방준비제도이사회(FRB)가 테이퍼링 계획을 추진하면서 인플레이션 위험 우려
7,How Retail Cheese Prices Weather Market Volati...,소매치즈 가격 변동성 예측 방법
8,How the Pandemic Is Accelerating Changes in Yo...,전염병이 포장의 변화를 가속화하는 방법
9,‘Dark Stores’: How Groceries Are Being Deliver...,'어두운 상점': 식료품이 몇 분 만에 배달되는 방법


In [58]:
df.to_csv('/content.drive/MyDrive/주가지수_1014/'+datetime.today().strftime("%Y-%m-%d")+'_headline.csv', encoding='utf-8')